# Example for utilization of powerfactory-tools -- Control

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import pathlib
import datetime as dt

from powerfactory_tools import PowerFactoryInterface
from powerfactory_tools.powerfactory_types import ResultExportMode
from powerfactory_tools.utils.io import FileType

In [ ]:
# Consider to use raw strings to avoid misinterpretation of special characters, e.g. r"dir\New Project" or r"dir\1-HV-grid".
PROJECT_NAME = "7 Tests\PowerFactory-Tools-Test"  # may be also full path "dir_name\project_name"
EXPORT_PATH = pathlib.Path("control_action_results")
PF_USER_PROFILE = "krahmers"  # specification may be necessary
PF_INI_NAME = ""  # optional specification of ini file name to switch to full version (e.g. PowerFactoryFull for file PowerFactoryFull.ini)
PYTHON_VERSION = "3.10"

## Control using a controller instance

In [ ]:
# Define element variables to monitor for later
element_vars = {
    "node": {
        "m:U1": "Uabs1",
        "m:U2": "Uabs2",
        "m:U0": "Uabs0",
        "m:phiu1": "Uang1",
        "m:phiu2": "Uang2",
        "m:phiu0": "Uang0",
    },
    "node_reduced": {
        "m:U1": "Uabs1",
        "m:U2": "Uabs2",
        "m:U0": "Uabs0",
    },
    "line": {
        "m:I1:bus1": "Iabs1",
        "m:I2:bus1": "Iabs2",
        "m:I0:bus1": "Iabs0",
        "m:phii1:bus1": "Iang1",
        "m:phii2:bus1": "Iang2",
        "m:phii0:bus1": "Iang0",
    },
}

export_data_name = "user_exported_data"
grid_name = "HV_9_Bus"

full_export_path = pathlib.Path().resolve() / EXPORT_PATH


with PowerFactoryInterface(
    project_name=PROJECT_NAME,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=PYTHON_VERSION,
    logging_level=logging.INFO,
) as pfi:
    ############
    ## Example 1: Request elements
    # Activate study case "Base"
    study_case = pfi.switch_study_case("Base")
    # Create new variant to work within (add second if one already exists with same name)
    grid_variant = pfi.create_grid_variant(name="control_example", force=True)
    if grid_variant:
        pfi.switch_grid_variant(grid_variant.loc_name)
    # Get grid object with specified name
    grid = pfi.grid(grid_name)
    # Get all objects from specific grid
    data = pfi.compile_powerfactory_data(grid)
    # All nodes in grid {grid_name}, also these that are out of service
    terminals_grid = data.terminals

    ############
    ## Example 2: Select special elements
    # All nodes of active study case which are in service and therefore calculation relevant
    terminals_study_case = pfi.terminals(calc_relevant=True)

    # Select only terminals with nominal voltage of xx kV
    voltage_threshold = 110
    terminals_sel = []  # selected terminals
    for term in terminals_study_case:
        # nominal voltage
        u_n = term.uknom
        if u_n == voltage_threshold:
            terminals_sel.append(term)

    ############
    ## Example 3: Change attribute values
    # Select some loads that names start with "Load"
    loads = pfi.loads("Load*", grid_name=grid_name)

    # Change power of loads
    for load in loads:
        load.plini = 2  # active power in MW

    ############
    ## Example 4: Create variable monitors (a selection of monitored variables for specific elements)
    ## Adapt default result object
    # Get existing default result object
    default_result = pfi.result("All*", study_case_name=study_case.loc_name)
    # Create variable monitor objects
    for term in terminals_sel:
        # Create variable monitor for each terminal
        pfi.create_variable_monitor(element=term, result=default_result, variables=element_vars["node"].keys())

    ## Create new result object
    # Create new result object
    new_result = pfi.create_result(name="New Results", study_case=study_case)
    # Create variable monitor objects
    for term in terminals_sel:
        # Create variable monitor for each terminal
        pfi.create_variable_monitor(element=term, result=new_result, variables=element_vars["node_reduced"].keys())

    ############
    ## Example 5: Run load flow and export results
    ## Run symmetrical AC load flow
    pfi.run_ldf(ac=True, symmetrical=False)

    ## Setup result export - variant I
    # a) Assign variable monitors to a result
    pfi.set_all_variable_monitors_as_result_variables(default_result)

    # b) Create result export command and assign the result objectand execute
    res_exp_cmd_1 = pfi.create_result_export_command(
        result=default_result,
        study_case=study_case,
        export_path=full_export_path,
        export_mode=ResultExportMode.CSV,
        name="My_Result_Export",
    )
    # Execute result export - variant I
    pfi.run_result_export(res_exp_cmd_1)

    ## Setup result export - variant II
    # a) Assign variable monitors to a result
    pfi.set_all_variable_monitors_as_result_variables(new_result)

    # b) Create result export command and execute
    res_exp_cmd_2 = pfi.create_result_export_command(
        result=new_result,
        study_case=study_case,
        export_path=full_export_path,
        export_mode=ResultExportMode.CSV,
        name="My_Result_Export_2",
    )

    # Execute result export - variant II
    pfi.run_result_export(res_exp_cmd_2)

    ## Export results - Variant III
    # Do further user specific work and fill result_data dictionary based on PF result
    data = {
        f"{terminals_sel[0].loc_name}": {
            "Uabs1": {
                "value": terminals_sel[0].GetAttribute("m:U1"),
            },
        },
    }
    # Store results
    pfi.export_data(
        data=data,
        export_path=full_export_path,
        file_type=FileType.JSON,
        export_data_name=export_data_name,
    )

    ###########
    ## Example 6: Run time domain simulations (RMS, EMT)
    rms_result = pfi.run_rms_simulation(time=3)

    # if rms simulation was successful, run result export using builtin function
    if rms_result:
        res_exp_cmd = pfi.create_result_export_command(
            result=rms_result,
            study_case=study_case,
            export_path=full_export_path,
            export_mode=ResultExportMode.CSV,
        )
        pfi.run_result_export(res_exp_cmd)

    # Run EMT simulation
    emt_result = pfi.run_emt_simulation(time=3)

    ############
    ## Example 7: Request and study cases, operation scenarios and network variants
    # Study Case
    study_cases = pfi.study_cases()  # get all
    study_case_active = pfi.study_case(only_active=True)  # get only active one

    # Network Variation
    variant = pfi.grid_variants()  # get all
    variants_active = pfi.grid_variants(only_active=True)  # get only active one(s)

    # Operation Scenarios
    scenarios = pfi.scenarios()  # get all
    scenario_active = pfi.scenario(only_active=True)  # get only active one

    ############
    ## Example 8: Create new grid variant, activate it, change topology within and deactivate
    pfi.switch_study_case("Base")

    # Create Grid Variant
    variant_folder = pfi.create_folder(name="user_variants", location=pfi.grid_variant_dir, update=True)
    variant = pfi.create_grid_variant(name="Variant1", location=variant_folder, update=True)

    # Activate this new grid variant
    pfi.activate_grid_variant(variant)

    # Set transformer out of service
    # if no scenario is active, changes regarding operation are stored in grid variant
    transformer = pfi.transformer_2w("Transformer_2w_110/20", grid_name=grid_name)
    transformer.outserv = 1

    # Set only one grid active
    pfi.deactivate_grids()
    pfi.activate_grid(pfi.grid(grid_name))

    # Deactivate all grid variants
    for variant in pfi.grid_variants(only_active=True):
        pfi.deactivate_grid_variant(variant)

    ############
    ## Example 9: Create new study case
    study_case = pfi.create_study_case(
        name="Industry_Park_v2",
        grids=pfi.independent_grids(),
        grid_variants=[variant],
        target_datetime=dt.datetime(1980, 1, 1, tzinfo=dt.timezone.utc),
    )

## Control using the control function running in a new process with default parameters

In [ ]:
import multiprocessing

POWERFACTORY_PATH = pathlib.Path("C:/Program Files/DIgSILENT")
POWERFACTORY_VERSION = "2022 SP2"


class PowerFactoryControllerProcess(multiprocessing.Process):
    def __init__(
        self,
        *,
        export_path: pathlib.Path,
        project_name: str,
        grid_name: str,
        export_data_name: str = "",
        powerfactory_user_profile: str = "",
        powerfactory_path: pathlib.Path = POWERFACTORY_PATH,
        powerfactory_version: str = POWERFACTORY_VERSION,
        powerfactory_ini_name: str = "",
        python_version: str = PYTHON_VERSION,
        logging_level: int = logging.INFO,
        log_file_path: pathlib.Path | None = None,
    ) -> None:
        super().__init__()
        self.export_path = export_path
        self.project_name = project_name
        self.grid_name = grid_name
        self.export_data_name = export_data_name
        self.powerfactory_user_profile = powerfactory_user_profile
        self.powerfactory_path = powerfactory_path
        self.powerfactory_version = powerfactory_version
        self.powerfactory_ini_name = powerfactory_ini_name
        self.python_version = python_version
        self.logging_level = logging_level
        self.log_file_path = log_file_path

    def run(self) -> None:
        pfi = PowerFactoryInterface(
            project_name=self.project_name,
            powerfactory_user_profile=self.powerfactory_user_profile,
            powerfactory_path=self.powerfactory_path,
            powerfactory_version=self.powerfactory_version,
            powerfactory_ini_name=self.powerfactory_ini_name,
            python_version=self.python_version,
            logging_level=self.logging_level,
            log_file_path=self.log_file_path,
        )
        ## Example 1: Request elements
        # Activate study case "Base"
        pfi.switch_study_case("Base")
        # Get grid object with specified name
        grid = pfi.grid(self.grid_name)
        # Get all objects from specific grid
        data = pfi.compile_powerfactory_data(grid)
        # All nodes in grid {grid_name}, also these that are out of service
        terminals_grid = data.terminals

        ############
        ## Example 2: Select special elements
        # All nodes of active study case which are in service and therefore calculation relevant
        terminals_study_case = pfi.terminals(calc_relevant=True)

        # Select only terminals with nominal voltage of xx kV
        voltage_threshold = 110
        terminals_sel = []  # selected terminals
        for term in terminals_study_case:
            # nominal voltage
            u_n = term.uknom
            if u_n == voltage_threshold:
                terminals_sel.append(term)

        ############
        ## Example 3: Change attribute values
        # Select some loads that names start with "Load"
        loads = pfi.loads(name="Load*", grid_name=self.grid_name)

        # Change power of loads
        for load in loads:
            load.plini = 2  # active power in MW

        ############
        ## Example 4: Run load flow - sym or unsym
        ldf_result = pfi.run_ldf(ac=True, symmetrical=False)
        # Do further user specific work and fill result_data dictionary based on PF result
        result_data = {}

        # Store results
        export_data(
            data=result_data,
            export_data_name=export_data_name,
            export_path=EXPORT_PATH,
        )

        ############
        ## Example 5: Request and study cases, operation scenarios and network variants
        # Study Case
        study_cases = pfi.study_cases()  # get all
        study_case_active = pfi.study_case(only_active=True)  # get only active one

        # Network Variation
        variant = pfi.grid_variants()  # get all
        variants_active = pfi.grid_variants(only_active=True)  # get only active one(s)

        # Operation Scenarios
        scenarios = pfi.scenarios()  # get all
        scenario_active = pfi.scenario(only_active=True)  # get only active one

        ############
        ## Example 6: Create new grid variant, activate it, change topology within and deactivate
        pfi.switch_study_case("Base")

        # Create Grid Variant
        variant_folder = pfi.create_folder(name="user_variants", location=pfi.grid_variant_dir, update=True)
        variant = pfi.create_grid_variant(name="Variant1", location=variant_folder, update=True)

        # Activate this new grid variant
        pfi.activate_grid_variant(variant)

        # Set transformer out of service
        # if no scenario is active, changes regarding operation are stored in grid variant
        transformer = pfi.transformer_2w("Transformer_2w_110/20", grid_name=grid_name)
        transformer.outserv = 1

        # Set only one grid active
        pfi.deactivate_grids()
        pfi.activate_grid(pfi.grid(grid_name))

        # Deactivate all grid variants
        for variant in pfi.grid_variants(only_active=True):
            pfi.deactivate_grid_variant(variant)

        ############
        ## Example 7: Create new study case
        study_case = pfi.create_study_case(
            name="Industry_Park_v2",
            grids=pfi.independent_grids(),
            grid_variants=[variant],
            target_datetime=dt.datetime(1980, 1, 1, tzinfo=dt.timezone.utc),
        )

In [ ]:
def apply_control(
    export_path: pathlib.Path,
    project_name: str,
    grid_name: str,
    export_data_name: str = "",
    powerfactory_user_profile: str = "",
    powerfactory_path: pathlib.Path = POWERFACTORY_PATH,
    powerfactory_version: str = POWERFACTORY_VERSION,
    powerfactory_ini_name: str = "",
    python_version: str = PYTHON_VERSION,
    logging_level: int = logging.INFO,
) -> None:
    process = PowerFactoryControllerProcess(
        project_name=project_name,
        export_path=export_path,
        grid_name=grid_name,
        export_data_name=export_data_name,
        powerfactory_user_profile=powerfactory_user_profile,
        powerfactory_path=powerfactory_path,
        powerfactory_version=powerfactory_version,
        powerfactory_ini_name=powerfactory_ini_name,
        python_version=python_version,
        logging_level=logging_level,
        log_file_path=pathlib.Path("pf_control.log"),
    )
    process.start()
    process.join()

### As the control function is executed in a process that is terminated after execution, the PowerFactory API is also closed.

In [ ]:
export_data_name = "export_data"
grid_name = "HV_9_Bus"

apply_control(
    export_path=EXPORT_PATH,
    project_name=PROJECT_NAME,
    grid_name=grid_name,
    export_data_name=export_data_name,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=PYTHON_VERSION,
)